<a href="https://colab.research.google.com/github/mastermedia092/crypto_balance_tool/blob/main/Analyze_Raydium_AMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Install packages

In [ ]:
# pip install solana

In [ ]:
# pip install plotly

# 1.Getting start, FILL IN YOUR ENDPOINT HERE 👇👇👇👇

In [ ]:
endPoint = ""

# 2. Initialization

In [ ]:
import json
import solana
import time
import logging
import threading
import numpy as np
import urllib.request
import json
from urllib.request import Request, urlopen

from solana.rpc.api import Client
explorerULRAdd = "https://explorer.solana.com/address/"
explorerULRTx = "https://explorer.solana.com/tx/"

Address_RaydiumAMM = "675kPX9MHTjS2zt1qfr1NYHuzeLXfQM9H24wFSUt1Mp8"
Address_liquidityPool = "5Q544fKrFoe6tsEbD7S8EmxGTJYAKtTVhAW5Q5pge4j1"
Address_solscan = "https://api.solscan.io/account?address="

In [ ]:
http_client = Client(endPoint)

# 3. Download Raydium data from solana

In [ ]:
lastSignature = None
resultArr = []
rounds = 0
txCount = 0
maxTxCount = 10000 #about 8 mins
Address_RaydiumAMM = "675kPX9MHTjS2zt1qfr1NYHuzeLXfQM9H24wFSUt1Mp8"

def getTxDetail(txSignature):
    txSignature2 = solana.transaction.Signature.from_string(txSignature)
    tx = http_client.get_transaction(txSignature2,max_supported_transaction_version=0)
    tx = json.loads(tx.to_json())
    postTokenBalances = tx["result"]["meta"]["postTokenBalances"]
    preTokenBalances = tx["result"]["meta"]["preTokenBalances"]
    if (postTokenBalances != preTokenBalances):
        print(explorerULRTx+txSignature)
        resultArr.append(tx)


# Getting all transactions
if __name__ == "__main__":
    RaydiumPubKey = solana.rpc.types.Pubkey.from_string(Address_RaydiumAMM)
    while(True):
        print("round-"+str(rounds+1))
        print("getting transactions")
        RaydiumPubKey = solana.rpc.types.Pubkey.from_string(Address_RaydiumAMM)
        txs = http_client.get_signatures_for_address(RaydiumPubKey,limit=200,before=lastSignature).to_json()
        txs = json.loads(txs) ["result"]
        print("processing signatures")
        signatures = np.array([o["signature"] for o in txs])

        threads = list()
        for signature in signatures:
            txCount += 1
            x = threading.Thread(target=getTxDetail, args=(signature,))
            threads.append(x)
            x.start()
        for index, thread in enumerate(threads):
            thread.join()
        if(txCount >= maxTxCount):
            break
        else:
            rounds += 1
            lastSignature = solana.transaction.Signature.from_string(txs[-1]["signature"])
        time.sleep(3)

Streaming output truncated to the last 5000 lines.
https://explorer.solana.com/tx/96AwxwAETQ4EcFNGrmN9mQEfmbi7ho5rmoeg3v1hC87f958gs14wq8GGChaZNcfTLDXecmdJhfRiyrMDnYJcL1zhttps://explorer.solana.com/tx/YM5LSuNfaF9ujbB8Lxq7aG9tvapMD2tm6fa1HQVipHEHfrbiit3XxtWyLr52dXGNKg4NTcwt9y3SFxjuHM4Mw3k

https://explorer.solana.com/tx/4aaKBJagiptZmg1mhAcajuQbJUu8q3FcaRCfB9QbMH3qyAyycLZnzxGnK2FYxrmsmCGfvm3uZmoNXJDNiRF3BbBC
https://explorer.solana.com/tx/3SLFGR1CQQtZqNzPrNh7ZTaNNJeiBchTz2CcRTzAu5mfJMy8MRH1baRLfrYL1YetRQe3Vi2CNY58WteYA45gTTKH
https://explorer.solana.com/tx/qXnoCVAfsGZ1FgYni5g6uUCANc3xF1VTbBeYjJXKezAD5WMrzatH6pDcPTBrAWhhQadKSRkPaYEmjjFH4YQqRCQ
https://explorer.solana.com/tx/RyH53JkMXV9YcEAdzeGwVYXRuckRn8HfwccXzEcUbJ7o6okM3dFy1aiqpSUq5ZATYFDfpECURSe9GTpC8aGebtV
https://explorer.solana.com/tx/M1AixP5ufwQfshrXXR995yuqs5Jp2zTMT9rEJZGpRcuPrmrsFEUZMFVpnEEGmB4k5KjofXG9eYKN4kApDthSqGZhttps://explorer.solana.com/tx/5emxZ1dCRkEWjojnyLU6rVadbCzE1weeGv8yB1zDFxnyDhRFcYApCw3HzPqFh5Cp3ZpsMouG3zyaRN1GY565

# 4. Porocess result and store in processedTxArr

In [ ]:
def processTx(tx):
    txSignature = tx["result"]["transaction"]["signatures"][0]
    txSender = tx["result"]["transaction"]["message"]["accountKeys"][0]
    blockTime = tx["result"]["blockTime"]
    slot = tx["result"]["slot"]
    postTokenBalances = tx["result"]["meta"]["postTokenBalances"]
    preTokenBalances = tx["result"]["meta"]["preTokenBalances"]
    tokenBalances = []
    for pre, post in zip(preTokenBalances, postTokenBalances):
        change = 0
        if not (post["uiTokenAmount"]["uiAmount"] is None or pre["uiTokenAmount"]["uiAmount"] is None):
            change = post["uiTokenAmount"]["uiAmount"] - pre["uiTokenAmount"]["uiAmount"]
        owner = pre["owner"]
        token = pre["mint"]
        if(change != 0):
            tokenBalances.append({
                "owner":owner,
                "token":token,
                "change":change
            })
    result = {
        "txSignature":txSignature,
        "sender" : txSender,
        "blockTime" : blockTime,
        "slot" : slot,
        "tokenBalances" : tokenBalances
    }
    return result

processedTxArr = []
for tx in resultArr:
    processedTxArr.append(processTx(tx))

In [ ]:
processedTxArr[0]

{'txSignature': '2qs5y6e5e9nLUTcQLD8rU1x2tgR1SHXwQnZJpzUciU8c3koEXUzSymziF1UintfVAZYfitgtwsLtprQf8tGUvRTs',
 'sender': 'EkZStqj9BSwLS19uLDEsErCW6N1HHzvoGg92Ei3YYBNt',
 'blockTime': 1687928543,
 'slot': 202136085,
 'tokenBalances': [{'owner': 'HobVrQpdkms7h57vcfaTW4Lqmxd5EatY2MYRavtvF1oL',
   'token': 'mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So',
   'change': -0.00016410899979746318},
  {'owner': 'EyaSjUtSgo9aRD1f8LWXwdvkpDTmXAW54yoSHZRF14WL',
   'token': 'mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So',
   'change': -0.0008310800000010943},
  {'owner': 'EkZStqj9BSwLS19uLDEsErCW6N1HHzvoGg92Ei3YYBNt',
   'token': 'mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So',
   'change': 0.0008359589999999972},
  {'owner': '5Q544fKrFoe6tsEbD7S8EmxGTJYAKtTVhAW5Q5pge4j1',
   'token': '2KE2UNJKB6RGgb78DxJbi2HXSfCs1EocHj4FDMZPr4HA',
   'change': -0.6004399999510497},
  {'owner': '5Q544fKrFoe6tsEbD7S8EmxGTJYAKtTVhAW5Q5pge4j1',
   'token': 'mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So',
   'change': 0.0001592300

# 5. Mapping token mint address to readable token names.

In [ ]:
mintDic = {}
for tx in processedTxArr:
    for tokenBalance in tx["tokenBalances"]:
        token = tokenBalance["token"]
        if token not in mintDic:
            req = Request(Address_solscan+token, headers={'User-Agent': 'Mozilla/5.0'})
            data = json.loads(urlopen(req).read())
            print(data["data"]["tokenInfo"]["name"])
            time.sleep(1)
            mintDic[token] = data["data"]["tokenInfo"]["name"]

for tx in processedTxArr:
    for tokenBalance in tx["tokenBalances"]:
        tokenName = mintDic[tokenBalance["token"]]
        if tokenName != "":
            tokenBalance["token"] = tokenName

Marinade staked SOL (mSOL)
Slow Protocol
USD Coin
PUGAI
HTO

Ethrix
USDT
AIO Exchange Token
Wrapped SOL
DGEN Token
Lido Staked SOL
Solanium
Gari

Bonk
Star Atlas

Serum
Laddercaster

ALL ART
Raydium

Walken
Froggo
BLUE
Samoyed Coin


Moon Roll Coin

POX Token

Solpad Finance


Honeyland

Realy Token
GMT

KING
ZEBEC

PRT (Parrot Protocol)

PAI (Parrot USD)

DeGenius
PEPE Cash
Solcasino Token
Project SEED Token
GST

Step
trollbox

SHIBONK

Wrapped ALEPH (Sollet)
CROWN Token
Helium Network Token
Guacamole
DeFi Land Gold

Boring Protocol
Tanox


BREAD
YoloNolo Coin
Wrapped Ether (Wormhole)

Solrise Finance

MonkeyBucks
Only1 (LIKE)
Bolt Token

Star Atlas DAO

Plutonian DAO
PRNT
Wrapped BNB (Wormhole)
ARB Protocol


Raydium LP Token V4 (RAY-SOL)
NANA Token
UXD Stablecoin

Baby Bonk Inu

Audius (Wormhole)
Fronk
Wrapped Ethereum (Sollet)


JPOOL Solana Token


LUV
SolChicks
Decimated
ROA CORE
Saber Protocol Token

DUST Protocol
Cryowar Token
Shadow Token
Atlas Dex

BANG
Raydium LP Token V4 (R

# 6. Normalise token transaction, save in normalisedTxArr

In [ ]:
normalisedTxArr = []

for tx in processedTxArr:
    for tokenChange in tx["tokenBalances"]:
        normalisedTxArr.append({
            'txSignature': tx["txSignature"],
             'sender': tx["sender"],
             'blockTime': tx["blockTime"],
             'slot': tx["slot"],
             'owner': tokenChange["owner"],
             'token': tokenChange["token"],
             'change': tokenChange["change"]
        })

In [ ]:
normalisedTxArr[0]

{'txSignature': '2qs5y6e5e9nLUTcQLD8rU1x2tgR1SHXwQnZJpzUciU8c3koEXUzSymziF1UintfVAZYfitgtwsLtprQf8tGUvRTs',
 'sender': 'EkZStqj9BSwLS19uLDEsErCW6N1HHzvoGg92Ei3YYBNt',
 'blockTime': 1687928543,
 'slot': 202136085,
 'owner': 'HobVrQpdkms7h57vcfaTW4Lqmxd5EatY2MYRavtvF1oL',
 'token': 'Marinade staked SOL (mSOL)',
 'change': -0.00016410899979746318}

# Analysis

## 1. Summary

In [ ]:
# 1st Summary
print("Total transactions:\n"+str(txCount))

successCount = len(resultArr)
print("Valid transactions:\n"+str(successCount))

print("Valid transaction rate:\n"+str(successCount/txCount*100)+"%")

Total transactions:
10000
Valid transactions:
4858
Valid transaction rate:
48.58%


## 2. Most transacted token top 20

In [ ]:
# count token txs
import pandas as pd
txDf = pd.DataFrame(normalisedTxArr, columns = normalisedTxArr[0].keys())

LPtxs = txDf[txDf.owner == Address_liquidityPool]
print("Top 20 token transactions:")
top20Tkn = LPtxs['token'].value_counts()[:20]
tempDf = pd.DataFrame({'name':top20Tkn.index, 'tx count':top20Tkn.values})

import plotly.express as px

fig = px.pie(tempDf,names = "name", values = "tx count", title='# of token transactions')
fig.show()

Top 20 token transactions:


## 3. Transaction volume of the top 5 tokens

In [ ]:
# token transaction volume
import pandas as pd
txDf = pd.DataFrame(normalisedTxArr, columns = normalisedTxArr[0].keys())

LPtxs = txDf[txDf.owner == Address_liquidityPool]
LPtxs["change"] = LPtxs["change"].abs()
top5Tkn = LPtxs['token'].value_counts()[:5]
tempdf = LPtxs.groupby(['token'])['change'].agg('sum')
tempdf = pd.DataFrame({'token':tempdf.index, 'change':tempdf.values})
result = tempdf.loc[tempdf['token'].isin(top5Tkn.keys()) ]
print("Transactions volume for top 5 tokens:")
print(result)

import plotly.express as px
fig = px.bar(result,x = "token", y = "change", title='Transactions volume for top 5 tokens:' , text_auto=True)
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

Transactions volume for top 5 tokens:
                                           token        change
17  CrYE3ngcy2VpTvhBb2rx3rSz1J2b5SpvCg5VTWytVXGr  4.259406e+08
44                    Marinade staked SOL (mSOL)  3.799738e+00
65                                 Slow Protocol  7.916073e+03
77                                      USD Coin  1.182551e+09
86                                   Wrapped SOL  5.064396e+06


<ipython-input-21-b8ff308bd82d>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## 4.Most active users

In [ ]:
# Top senders
import pandas as pd
txDf = pd.DataFrame(normalisedTxArr, columns = normalisedTxArr[0].keys())
txDf2 = pd.DataFrame(processedTxArr, columns = processedTxArr[0].keys())

top20Senders = txDf2['sender'].value_counts()[:20]
top20Senders = pd.DataFrame({'sender':top20Senders.index, 'count':top20Senders.values})

def getTokenExchangeCount(row):
    size = txDf[(txDf.owner == row.sender) & (txDf.owner== row.sender)].size
    return size
top20Senders['tokenExchangeCount'] = top20Senders.apply(lambda row: getTokenExchangeCount(row), axis=1)

print("Top 20 senders:")
print(top20Senders)

import plotly.express as px
fig = px.bar(top20Senders,x = "sender", y = "count", title='Most active sender address:' , text_auto=True)
fig.show()


Top 20 senders:
                                          sender  count  tokenExchangeCount
0   EkZStqj9BSwLS19uLDEsErCW6N1HHzvoGg92Ei3YYBNt   1734               12138
1    MeowujaCA1FGaA7hna2ezUXLbrj36qZi4hGx94xNQLo    391                2723
2   AuN1J75bHWiPD8ufmQc3LfM8roiPKrvjjjQ1mCQkqrkv    178                1330
3   77vqGeWLBTHAFM4FBjnjSvrN5mnvrWuFFbo1NBhcaaHZ    110                1540
4   Ej2Q8UKttPHDDTpP1f85c9gSo4byRYvdcbmRTthfoNyw    109                1526
5    dSX9LnEk9psuRmnQQi5BNmDVuzjLgkBbuwHoCF9qdKc    108                1512
6   9W8QVHh4Pgk4dmee3Cgm2weKEXLKbJWvHQfhNSLLkWVg     96                1344
7    degenv1vuXTFPuMsHHFwtCkovV3j1XmHNHnkQFhvkZ1     88                1232
8   AcBvyRVZJduyASUhhtyFzEuvSxSfAMBeRdW9t3Ajjdug     71                 994
9   8pjPBtmNuZfAYPyrtkU6nHX1BXMhRu8HicSJ6a4fujSb     64                 896
10  4KVBskF8LWptkBCC1Ldj98Ym7ndHhvVfnwS3AJDwUSzw     61                 441
11  FBggTQtYvKSgRFhQXfk5M1CgFBZa7btgYrtXtM834KR4     53                 

## 5.List top sender's transactions

In [ ]:
#@title
# Dig into sender's transaction type
import pandas as pd
txDf = pd.DataFrame(normalisedTxArr, columns = normalisedTxArr[0].keys())

senderHash = top20Senders.iloc[1]["sender"]
txDf[(txDf.owner == senderHash) & (txDf.sender == senderHash)].size
tempdf = txDf[(txDf.owner == senderHash) & (txDf.sender == senderHash)]
tempdf

,txSignature,sender,blockTime,slot,owner,token,change
53,5emxZ1dCRkEWjojnyLU6rVadbCzE1weeGv8yB1zDFxnyDh...,MeowujaCA1FGaA7hna2ezUXLbrj36qZi4hGx94xNQLo,1687928538,202136074,MeowujaCA1FGaA7hna2ezUXLbrj36qZi4hGx94xNQLo,Marinade staked SOL (mSOL),0.001924
110,GbmLDPJHALDodcZAXtvssTRvQvAUWeNyuzAcybypLtztMt...,MeowujaCA1FGaA7hna2ezUXLbrj36qZi4hGx94xNQLo,1687928534,202136065,MeowujaCA1FGaA7hna2ezUXLbrj36qZi4hGx94xNQLo,Marinade staked SOL (mSOL),0.001932
126,5yPKdfwY4coG1kT27bx3j6t6dQgEME4vV26U8GHFruvGuj...,MeowujaCA1FGaA7hna2ezUXLbrj36qZi4hGx94xNQLo,1687928527,202136050,MeowujaCA1FGaA7hna2ezUXLbrj36qZi4hGx94xNQLo,Marinade staked SOL (mSOL),0.001948
134,63EHojWguZ5LgzqwSiputz5pm6ivfGTq8f5GGaqXxp1p5x...,MeowujaCA1FGaA7hna2ezUXLbrj36qZi4hGx94xNQLo,1687928530,202136056,MeowujaCA1FGaA7hna2ezUXLbrj36qZi4hGx94xNQLo,Marinade staked SOL (mSOL),0.001925
139,5bhkY8hYTMUYnpe7eENb5stRKX6SKm9tkUFgyuf3zf2JLs...,MeowujaCA1FGaA7hna2ezUXLbrj36qZi4hGx94xNQLo,1687928529,202136053,MeowujaCA1FGaA7hna2ezUXLbrj36qZi4hGx94xNQLo,Marinade staked SOL (mSOL),0.001936
...,...,...,...,...,...,...,...
22976,5bxrBh8ccX256XvBkk5xtedwZJkZhX8cZfWPCRu3TGBqG1...,MeowujaCA1FGaA7hna2ezUXLbrj36qZi4hGx94xNQLo,1687926339,202131072,MeowujaCA1FGaA7hna2ezUXLbrj36qZi4hGx94xNQLo,Marinade staked SOL (mSOL),0.001926
23000,4qFX1hTsDFtA8eT1p6qW4TVCKxJAkj7Cy51o9Uu2NxeLrT...,MeowujaCA1FGaA7hna2ezUXLbrj36qZi4hGx94xNQLo,1687926335,202131064,MeowujaCA1FGaA7hna2ezUXLbrj36qZi4hGx94xNQLo,Marinade staked SOL (mSOL),0.001931
23013,2jPZXjJC9xkKjtNmYF1bJp5gpMY7VcU57597EL9b19MRy8...,MeowujaCA1FGaA7hna2ezUXLbrj36qZi4hGx94xNQLo,1687926334,202131060,MeowujaCA1FGaA7hna2ezUXLbrj36qZi4hGx94xNQLo,Marinade staked SOL (mSOL),0.001938
23024,N4pxavEswvfbMAyTZuxvr6jnsroTM8WUCscEZb1LBwFQMM...,MeowujaCA1FGaA7hna2ezUXLbrj36qZi4hGx94xNQLo,1687926332,202131057,MeowujaCA1FGaA7hna2ezUXLbrj36qZi4hGx94xNQLo,Marinade staked SOL (mSOL),0.001950


## 6. Identify sandwich attack suspect

In [ ]:
#@title
case_arr = []
n = len(processedTxArr)

# Identify "breads" in sandwich attack
for i in range(n-1):
    for j in range(i+1,n-1):
        for tx in processedTxArr[i]["tokenBalances"]:
            if(tx["owner"] == Address_liquidityPool or tx["change"] == 0):
                break
            for tx2 in processedTxArr[j]["tokenBalances"]:
                if(tx2["owner"] == Address_liquidityPool or tx2["change"] == 0):
                    break
                if(tx["owner"] == tx2["owner"] and tx["token"] == tx2["token"] and tx["change"] == -tx2["change"]):
                    owner = tx["owner"]
                    token = tx["token"]
                    amount = tx["change"]
                    txAdd1 = processedTxArr[i]["txSignature"]
                    txAdd2 = processedTxArr[j]["txSignature"]
                    potential_target_list = []

                    for tempTx in processedTxArr[i+1:j]:
                        potential_target_list.append(tempTx["txSignature"])

                    case_arr.append({
                        "suspect":owner,
                        "token":token,
                        "amount":abs(amount),
                        "suspicious_tx1":txAdd1,
                        "suspicious_tx2":txAdd2,
                        "suspicious_tx_index":i,
                        "suspicious_tx2_index":j,
                        "potential_target_tx":potential_target_list
                    })
                    print("Found suspicious transaction pair:")
                    print("Sender account:" + tx["owner"])
                    print("Token:" + tx["token"])
                    print("Amount exchanged:")
                    print(str(tx["change"])+","+str(tx2["change"]))
                    print("Tx addresses:")
                    print("https://solscan.io/tx/" + processedTxArr[i]["txSignature"])
                    print("https://solscan.io/tx/" + processedTxArr[j]["txSignature"])
                    print("Potential target:")
                    for tempTx in potential_target_list:
                        print("https://solscan.io/tx/" + tempTx)
                    print("---")

Found suspicious transaction pair:
Sender account:AcBvyRVZJduyASUhhtyFzEuvSxSfAMBeRdW9t3Ajjdug
Token:Wrapped SOL
Amount exchanged:
-2.4500000006533185e-07,2.4500000006533185e-07
Tx addresses:
https://solscan.io/tx/4YMh1APJ5DZorQBZpqC4GoL9FochiPHbt4ymXNSiy3VzP3sVPw7jG8t7fui6jSeYhSEivyPfCos417Fq46ex9aX5
https://solscan.io/tx/2qZNSKCwSB9RB3sAHPS1mpR7unhC4hZZUk1DeywZUR4sMCSXtrmBbnNSLJM5JUPGBrLqTHmwGzZUraW6zJMT33M1
Potential target:
https://solscan.io/tx/5DiFHfmmQPvaiGHnSfbQYrxgVKeb1fY9Ph6tLjBVzs9P4fMiFKPuvj2oCpcktPZvDVyJ1LuyG8dYpwuN6oXJQwJn
https://solscan.io/tx/5Vqry9Z14snVhx1FCZuNmSkFKX4LGruUoUzaUoPFKwzxcnpSmidt1FLd9qwuEF24PxW5vSBHd7kmLNEowZCvCKMB
https://solscan.io/tx/4js3UEojdjL4PfF7Q4ikYWyP8ntNg8b1xnwWZ2GPpbRD36akQjNLWEKsURCMxCnpMsQ5SmC8EehfPLZjNrrKTZTK
https://solscan.io/tx/5pFemffwWgkEvmYgQcfdGrvqefLmudmqBBNKB7rM1mUWVqosb4fpSkGu2Ng9sS8QeSDXohrifnv3xFgXY3xuFfNY
https://solscan.io/tx/4ivhKnNnU7RN8yd4eGLJeRbRQoPAsLS5vwRu5Cj7zGiV2E7k57QCY6Sxh89Dz1HwjsMS1qRFVEFXvA4UuipeSA59
https://solsc

Found suspicious transaction pair:
Sender account:Hh7LU33kJhJmnws22RPyCY6NkvSCvL9YEntw6zJiVWoc
Token:Raydium
Amount exchanged:
-10.308676,10.308676
Tx addresses:
https://solscan.io/tx/5DoA8mdhb1GQFGKbdsR8zsJbNHoMDPWwJXStn4haCtWLUCPFwh841GUn11F9hjzvfWiyPWJ9GncJjx14SM6f7D7C
https://solscan.io/tx/dr1SYktYnvC2gGmTboznf28q1CDrXJzwBELmPi2CG1cqJ3iwR9Nbv9nqjQPkvuJ62yaPr6V2qbHh89wkhg9zmjC
Potential target:
https://solscan.io/tx/Ugiqmi99gqYg2mKjqFaE7RJC149vjDorqvAiMHqxyQStNEo9AypaTyzygCqL6oJg8w8oT5uhUdvT3KyGCvpD9te

---
Found suspicious transaction pair:
Sender account:9Jbh6bcHxgxb7S1FfNjFTUQABmd2cwzHWY3gxG4A2CjD
Token:USD Coin
Amount exchanged:
7.046785,-7.046785
Tx addresses:
https://solscan.io/tx/3mfv7XHCY2g6GpDYBtEz41dtQr3Ccik9NtBwSd6BA97NJqDBVvzLmF9nnju5UDn3QmGuP5rwXjd8tRpUvUWw4GSD
https://solscan.io/tx/2xkWdixXqM8rtWPCLbLKXrVGJud4eKi4mseokQRXcpsM1MYh9H8ygeCbUTfaMZgsaEUdxUndFb3kPWz7fcdfmuKd
Potential target:

---